In [1]:
import torch
import numpy as np
import torch.nn as nn

In [78]:
class GaussianNoise(nn.Module):
    """Gaussian noise regularizer.

    Args:
        sigma (float, optional): relative standard deviation used to generate the
            noise. Relative means that it will be multiplied by the magnitude of
            the value your are adding the noise to. This means that sigma can be
            the same regardless of the scale of the vector.
        is_relative_detach (bool, optional): whether to detach the variable before
            computing the scale of the noise. If `False` then the scale of the noise
            won't be seen as a constant but something to optimize: this will bias the
            network to generate vectors with smaller values.
    """

    def __init__(self, sigma=0.1, is_relative_detach=False, inference=False, white=False):
        super().__init__()
        self.sigma = sigma
        self.is_relative_detach = is_relative_detach
        self.noise = torch.tensor(0).to("cuda")
        self.inference = inference
        self.white = white

    def forward(self, x):
        if (self.training and self.sigma > 0) or self.inference:
            scale = self.sigma * x.detach() if self.is_relative_detach else self.sigma * x
            if not self.white:
                sampled_noise = self.noise.repeat(*x.size()).float().normal_() * scale
            else:
                sampled_noise = self.noise.repeat(*x.size()).float().random_(-1000000,1000000) * self.sigma / 1000000
            x = x + sampled_noise
        return x 

In [79]:
model = GaussianNoise(sigma=0.05, is_relative_detach=True,)

In [85]:
# データのパラメータ
fs = 16384
N = fs
dt = 1/fs
f1 = 1000    # freq
t = np.arange(0, fs*dt, dt) # time
freq = np.linspace(0, 1, fs)*fs # fft bins

# offset
offset = 1/(2^12)

# signal generation
input = np.sin(2*np.pi*f1*t)*2

In [86]:
input.std()

1.4142135623730945

In [87]:
out = model.forward(torch.from_numpy(input).cuda()).cpu().numpy()

In [88]:
F = np.fft.fft(out)
# Spectrum analyze
N = len(input)
Amp = np.power(np.abs(F)[0:int(N/2)-1], 2)
Amp[0] = 0 # cut DC
freq = np.linspace(0, 1, N)*N # 周波数軸
freq = freq[0:int(N/2)-1]
#Amp = np.abs(F)
#freq = freq

# normalize
Amp /= max(Amp)

In [89]:
# SNR calc
sig_bin = np.where(Amp==np.abs(Amp).max())[0]
signal_power = Amp[sig_bin]

noise_power = Amp.sum() - signal_power

SNDR = signal_power / noise_power
SNDR = 10*np.log10(SNDR)

ENOB = (SNDR-1.76) / 6.02
print("SNDR:", SNDR)
print("ENOB:", ENOB)

SNDR: [25.98198716]
ENOB: [4.02358591]
